In [ ]:
import os
import re
import json
import random

import pandas as pd
from pyarrow import parquet as pq

In [2]:
# Function to read parquet file
def read_parquet_file(file_path: str) -> pd.DataFrame:
    table = pq.read_table(file_path)
    return table.to_pandas()

In [3]:
os.getcwd()

'/workspace'

In [4]:
os.chdir('/workspace/koko/myllm')
os.getcwd()

'/workspace/koko/myllm'

In [ ]:
import utils.clean as clean

# Load Json Data

In [8]:
path = "datasets/tokenizer/raw/minimind_tokenizer_train.jsonl"

# Load the data and convert to raw text
data = []
with open(path, 'r') as f:
    for line in f:
        data.append(json.loads(line)['text'])
len(data)

600000

# Load Parquet

In [9]:
parquet_file_path = [
    'datasets/pretrain/computer_programming_code/chinese/high/rank_00000_00000.parquet', 
    'datasets/pretrain/computer_programming_code/english/high/rank_00000_00000.parquet', 
    'datasets/pretrain/computer_programming_code/english/high/rank_00000_00001.parquet', 
    'datasets/pretrain/artificial_intelligence_machine_learning/english/high/rank_00000_00000.parquet', 
    'datasets/pretrain/artificial_intelligence_machine_learning/english/high/rank_00000_00001.parquet', 
    'datasets/pretrain/artificial_intelligence_machine_learning/english/high/rank_00000_00002.parquet', 
    'datasets/pretrain/artificial_intelligence_machine_learning/english/high/rank_00000_00003.parquet', 
]
parquet_dfs = [read_parquet_file(file_path) for file_path in parquet_file_path]

In [10]:
# traverse the parquets and get the text
for parquet_df in parquet_dfs:
    data.extend(parquet_df['text'].tolist())
len(data)

962313

# Clean Up data

In [13]:
parquet_dfs[1].text

0        Writing multi-threaded applications      {#mul...
1        Bloog Bot\n\nOpening Our Eyes\n\nIn Part 2 we ...
2        Q:\n\nHow can I compress a sequence of integer...
3        Q:\n\nDifference between Sum and Aggregate in ...
4        The State of the Language: An Interview with B...
                               ...                        
94944    Matrix algebra and manipulation\nProgram contr...
94945    Optimization Community Mourns Loss of Two Line...
94946    Software Simply\nsoftware development, functio...
94947    Java Put on a Regular Release Schedule\nThe JC...
94948    Java Put on a Regular Release Schedule\nThe JC...
Name: text, Length: 94949, dtype: object

In [15]:
data = pd.DataFrame(data, columns=['text'])
data.head()

,text
0,好的。现在请你将这个文本中的所有的逗号都替换成空格。 好的，请稍等一下，现在我会将文本中的所...
1,帮我回答一道历史题目。清朝时期的八旗共有多少旗人？ 清朝时期八旗旗人总数约为200万人左右，...
2,嗯，谢谢你介绍的做法很详细，但我不喜欢吃鸡蛋，有没有其他菜做法能介绍一下？ 当然，你可以试试...
3,请描述一下如何正确规划个人理财。 正确规划个人理财需要以下几个步骤：\n1.了解自己的财务状...
4,描述一下天堂和地狱的生态系统和环境。 天堂和地狱被认为是灵性信仰中关于死后世界的两种不同概念...


In [16]:
# Clean the data
data['text'] = data['text'].apply(clean.replace_consecutive_punctuation)
data['text'] = data['text'].apply(clean.replace_white_space)

In [17]:
# Convert to raw text
data = data['text'].tolist()

# Save the data as raw text

In [18]:
with open('datasets/tokenizer/processed/data.jsonl', 'w', encoding='utf8') as f:
    for text in data:
        f.write(json.dumps({'text': text}) + '\n')